In [2]:
from PCSampler import PCSampler, vis_pc, transform_cloud
from manifpy import SE3, SO3, SE3Tangent
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [14]:
S = PCSampler()
S.load_samples()
clouds = S.get_clouds()
transforms = S.get_transforms(use_quat=True)

In [15]:
#get the two first clouds and their tranformations to world
c0, c1 = clouds[0], clouds[8]
T0, T1 = SE3(transforms[0]), SE3(transforms[8])
c0_world = transform_cloud(c0, T0.transform())
c1_world = transform_cloud(c1, T1.transform())
#vis_pc([c0_world, c1_world])

In [28]:
#get ground truth relative transformation
T = T0.inverse()*T1
c1_0 = transform_cloud(c1, T.transform())
vis_pc([c0, c1_0])


In [37]:
#give T some random perturbations to simulate odometry noise
cov_pos = 10
cov_yaw = 0.01
p = np.zeros(6)
p[:2] = np.sqrt(cov_pos)*np.random.normal(size=2) # we only perturb x,y
p[5] = np.sqrt(cov_yaw)*np.random.normal(size=1) # we only perturb yaw
T_p = SE3Tangent(p) + T #I guess we want to add p on the left side?
c1_0p = transform_cloud(c1, T_p.transform())
vis_pc([c0, c1_0p])

In [27]:
#convert clouds to open3d clouds
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(c0)
pcd0.estimate_normals()

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(c1)
pcd0.estimate_normals()

#do registration
treshold = 100
initial_T = T_p.transform()
reg = o3d.pipelines.registration.registration_icp(
    pcd1, pcd0, treshold, initial_T, 
    o3d.pipelines.registration.TransformationEstimationPointToPlane()
)
icp_transform = reg.transformation
print(icp_transform)
c1_registered = transform_cloud(c1, icp_transform)

#vis_pc([c0, c1_0])
vis_pc([c0, c1_registered])
print(reg)
print(T.transform())

[[ 9.96959074e-01  7.74747512e-02  8.38257738e-03  6.69788712e+00]
 [-7.74625447e-02  9.96993689e-01 -1.77166479e-03  4.99092154e-01]
 [-8.49463603e-03  1.11694151e-03  9.99963296e-01 -1.73756717e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
RegistrationResult with fitness=1.000000e+00, inlier_rmse=4.159245e-01, and correspondence_set size of 6294
Access transformation to get result.
[[ 9.97098156e-01  7.61241395e-02  6.18627839e-04  8.37998950e+00]
 [-7.61241781e-02  9.97098344e-01  3.90736383e-05  5.53731008e-01]
 [-6.13858347e-04 -8.60527886e-05  9.99999808e-01 -7.78385706e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [26]:
T = SE3()
T.setIdentity()
l = np.ones(6)
t = T.log()
l += t.coeffs()
print(type(t.coeffs()))

<class 'numpy.ndarray'>
